# Test trial balance data in database

In [18]:
import pandas as pd
import sqlite3

from luca import p, chart_of_accounts_from_db, journal_from_db, TrialBalance

# Corrections
Some of the data has been input incorrectly and needs correcting.  This is safe to repeat.

In [3]:
con = sqlite3.connect("historic_trial_balances.db")
cursor = con.cursor()
for c in [('YTD-DEC-15', 6100, 15590.9, ),  # Error in Excel spreadsheet
          ('YTD-DEC-15', 8414, 1547.4, ),   # Error in Excel spreadsheet
          ('YTD-JAN-15', 1001, 362381.70, ),  # Unknown error changing fomr 364193.12 to make BS balance
         ]:
    sql_d = "DELETE FROM trial_balance WHERE period = '{}' and code = {}".format(c[0], c[1]) # no unique index so delete 
    print(sql_d)
    cursor.execute(sql_d)
    #  multiple versions
    sql = "INSERT INTO trial_balance VALUES ('{}', {}, {})".format(c[0], c[1], c[2])
    print(sql)
    cursor.execute(sql)
con.commit()
con.close()


DELETE FROM trial_balance WHERE period = 'YTD-DEC-15' and code = 6100
INSERT INTO trial_balance VALUES ('YTD-DEC-15', 6100, 15590.9)
DELETE FROM trial_balance WHERE period = 'YTD-DEC-15' and code = 8414
INSERT INTO trial_balance VALUES ('YTD-DEC-15', 8414, 1547.4)
DELETE FROM trial_balance WHERE period = 'YTD-JAN-15' and code = 1001
INSERT INTO trial_balance VALUES ('YTD-JAN-15', 1001, 362381.7)


# Investigate data in database to see if ok

In [5]:
dbname='historic_trial_balances.db'

In [19]:
with chart_of_accounts_from_db(dbname) as coa_s:
    coa = coa_s.get_chart_of_account('SLF-MA')

In [26]:
con = sqlite3.connect("historic_trial_balances.db")
sql = "SELECT * FROM trial_balance"
df2 = pd.read_sql(sql, con)
con.close()
df2.tail()
periods = list(set(df2['period'].tolist()))
periods.sort()
df = pd.DataFrame(periods, index=periods, columns=['Period'])
df.head()

,Period
AMTD-APR-15,AMTD-APR-15
AMTD-APR-16,AMTD-APR-16
AMTD-AUG-15,AMTD-AUG-15
AMTD-AUG-16,AMTD-AUG-16
AMTD-DEC-15,AMTD-DEC-15


In [27]:
result = []
sales = []
for i, row in enumerate(df.iterrows()):
    if i < 4:
        print(row[0])
    with journal_from_db(dbname, coa, journal_entry_class=TrialBalance) as js:
        tb = js.get_entry(row[0])  # YTD-FEB-15
    result.append(tb.sum())
    sales.append(tb[4000])
df['Sum']=result
df['Sales']=sales
df.head(20)

AMTD-APR-15
AMTD-APR-16
AMTD-AUG-15
AMTD-AUG-16


,Period,Sum,Sales
AMTD-APR-15,AMTD-APR-15,0.00,-390552.68
AMTD-APR-16,AMTD-APR-16,0.00,-205602.75
AMTD-AUG-15,AMTD-AUG-15,0.00,-974147.29
AMTD-AUG-16,AMTD-AUG-16,0.00,-205602.75
AMTD-DEC-15,AMTD-DEC-15,0.00,-1496178.37
AMTD-DEC-16,AMTD-DEC-16,0.00,-205602.75
AMTD-FEB-15,AMTD-FEB-15,0.00,-156580.95
AMTD-FEB-16,AMTD-FEB-16,0.00,-82969.42
AMTD-JAN-16,AMTD-JAN-16,0.00,-1656832.85
AMTD-JAN-17,AMTD-JAN-17,0.00,-205602.75


In [28]:
df.ix[['AMTD-JAN-16', 'AYTD-JAN-16']]

,Period,Sum,Sales
AMTD-JAN-16,AMTD-JAN-16,0.00,-1656832.85
AYTD-JAN-16,AYTD-JAN-16,0.00,-1656832.85


In [41]:
for period in ('OCT-14', 'NOV-14', 'MAR-15', 'APR-15', 'MAY-15', 'DEC-15'):
    print('-----')
    print('>> {}'.format(period))
    for prefix in ('MTD-', 'YTD-', ):
        tb = prefix + period
        con = sqlite3.connect("historic_trial_balances.db")
        sql = "SELECT code as Code, balance as TB FROM trial_balance WHERE period = '{}'".format(tb)
        df1 = pd.read_sql(sql, con, index_col='Code')
        con.close()
        print('{} {}'.format(tb, p(df1['TB'].sum())))
        # print(df1)

-----
>> OCT-14
MTD-OCT-14 -4425.40
YTD-OCT-14 -845.03
-----
>> NOV-14
MTD-NOV-14 -3933.83
YTD-NOV-14 -0.08
-----
>> MAR-15
MTD-MAR-15 15018.34
YTD-MAR-15 4299.19
-----
>> APR-15
MTD-APR-15 14638.51
YTD-APR-15 973.55
-----
>> MAY-15
MTD-MAY-15 0.00
YTD-MAY-15 0.00
-----
>> DEC-15
MTD-DEC-15 6837.50
YTD-DEC-15 -799.00


In [29]:
p(df1['TB'].sum())

Decimal('-6726.90')

In [39]:
print(df1)

              TB
Code            
4000 -1496178.37
4009    34550.90
4905    35866.66
5000   920189.26
6200    26024.18
6201     6213.85
7000   106124.34
7006        0.00
7020        0.00
7100     8250.00
7102    29935.53
7103     1038.41
7105    10143.92
7200      105.00
7202    10120.00
7204    83511.15
7206    16310.95
7906        0.00
8100        1.21
8200     3300.00
8201     1123.50
8204     1713.29
8300     2020.00
8310        0.00
8400     1375.00
8402     5262.99
8405    10243.02
8408      894.44
8410    22220.00
8414     4547.40
8420   135545.57
8424     2769.57
8426     7966.54
8430     6869.37
8433     1235.00
8435     1506.38
8440     6218.89
9999        0.00
10      10613.54
1001   235986.42
1100   277026.42
1102        0.00
1103    31181.08
1115    67841.16
1200    87485.92
1202    58270.26
1203        0.00
1204      586.12
2100   -13480.90
2103  -231405.93
2104    26017.94
2105        0.00
2106        0.00
2107    -9308.70
2108   -28746.76
2109    26316.38
2110        0.

In [12]:
df1[df1.index > 3000].sum()

TB    13680.95
dtype: float64